In [43]:
# Impoeted modules
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [45]:
# created fucntions

def groupby_mean(x):
    return x.mean()

def groupby_count(x):
    return x.count()

def purchase_duration(x):
    return (x.max() - x.min()).days

def avg_frequency(x):
    ''' average time (days) between sales'''
    return (x.max() - x.min()).days/x.count()

    #adding __name__ so we can refer to it by single name

groupby_mean.__name__ =  "avg"
groupby_count.__name__ = "count"
purchase_duration.__name__ = "purch_dur" #fightning strong urge to call it "pur_dur"....
avg_frequency.__name__ = "purch_freq"



In [2]:
file_name = "https://raw.githubusercontent.com/rajeevratan84/datascienceforbusiness/master/OnlineRetail.xlsx"
df = pd.read_excel(file_name, sheet_name='Online Retail')

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.shape

(541909, 8)

In [4]:
df = df.loc[df['Quantity'] > 0 ] #removing 0 quantities

In [5]:
df = df[pd.notnull(df['CustomerID'])]

# Data Cleaning and EDA 

In [6]:
print('Date range: %s to %s' % (df['InvoiceDate'].min(), df['InvoiceDate'].max()))

Date range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00


In [7]:
df = df.loc[df["InvoiceDate"] < '2011-12-01']

In [8]:
df['Sales'] = df['Quantity'] * df['UnitPrice']

In [41]:
orders = df.groupby(['CustomerID', 'InvoiceNo']).agg({
    'Sales' : "sum",
    'InvoiceDate' : 'max'
})

In [42]:
orders.head()

Sales         InvoiceDate
CustomerID InvoiceNo                              
12346.0    541431     77183.60 2011-01-18 10:01:00
12347.0    537626       711.79 2010-12-07 14:57:00
           542237       475.39 2011-01-26 14:30:00
           549222       636.25 2011-04-07 10:43:00
           556201       382.52 2011-06-09 13:01:00

In [48]:
summary = orders.reset_index().groupby('CustomerID').agg({
    'Sales' : ["min","max","sum", groupby_mean, groupby_count],
    'InvoiceDate' : ["min","max",purchase_duration , avg_frequency]
})

In [49]:
summary.head()

Sales                                          \
                 min       max       sum           avg count   
CustomerID                                                     
12346.0     77183.60  77183.60  77183.60  77183.600000   1.0   
12347.0       382.52   1294.32   4085.18    680.863333   6.0   
12348.0       227.44    892.80   1797.24    449.310000   4.0   
12349.0      1757.55   1757.55   1757.55   1757.550000   1.0   
12350.0       334.40    334.40    334.40    334.400000   1.0   

                   InvoiceDate                                           
                           min                 max purch_dur purch_freq  
CustomerID                                                               
12346.0    2011-01-18 10:01:00 2011-01-18 10:01:00         0        0.0  
12347.0    2010-12-07 14:57:00 2011-10-31 12:25:00       327       54.5  
12348.0    2010-12-16 19:09:00 2011-09-25 13:13:00       282       70.5  
12349.0    2011-11-21 09:51:00 2011-11-21 09:51:00         0        0.0  
12350.0    2011-02-02 16:01:00 2011-02-02 16:01:00         0        0.0

In [50]:
summary.columns = ["_".join(col).lower() for col in summary.columns]

In [52]:
summary.columns

Index(['sales_min', 'sales_max', 'sales_sum', 'sales_avg', 'sales_count',
       'invoicedate_min', 'invoicedate_max', 'invoicedate_purch_dur',
       'invoicedate_purch_freq'],
      dtype='object')